In [1]:
# import capytaine as cpy
import autograd.numpy as np
import matplotlib.pyplot as plt
import xarray as xr
# from math import comb

import wecopttool as wot
import os
import Pioneer_Inverted_Pendulum as pip

## set colorblind-friendly colormap for plots
plt.style.use('tableau-colorblind10')

wot.set_loglevel('INFO', capytaine=False)
data_path = os.path.join('pioneer_data')

In [2]:
wavefreq = 0.325
f1_reg = wavefreq
nfreq_reg = 5
amplitude = 0.15

waves_regular = wot.waves.regular_wave(f1_reg, nfreq_reg, wavefreq, amplitude)

In [3]:
omega = waves_regular.omega

In [4]:
NPIP = pip.NonlinearInvertedPendulumPTO(f1 = f1_reg,
                                        nfreq = nfreq_reg,
                                        ndof = 1,
                                        control_type = 'PI')

In [5]:
NPIP.nstate_opt

12

In [6]:
NPIP.f_add

{'Generator': <bound method NonlinearInvertedPendulumPTO.torque_from_PTO of <Pioneer_Inverted_Pendulum.NonlinearInvertedPendulumPTO object at 0x0000019154815250>>,
 'Friction': <bound method NonlinearInvertedPendulumPTO.nonlinear_torque_from_friction of <Pioneer_Inverted_Pendulum.NonlinearInvertedPendulumPTO object at 0x0000019154815250>>,
 'Spring': <bound method NonlinearInvertedPendulumPTO.torque_from_nl_spring of <Pioneer_Inverted_Pendulum.NonlinearInvertedPendulumPTO object at 0x0000019154815250>>,
 'Pendulum': <bound method NonlinearInvertedPendulumPTO.torque_from_pendulum of <Pioneer_Inverted_Pendulum.NonlinearInvertedPendulumPTO object at 0x0000019154815250>>}

In [7]:
wec_bem = pip.PioneerBuoy.from_bem_data(f1_reg, nfreq_reg)
wec_emp = pip.PioneerBuoy.from_empirical_data(f1_reg, nfreq_reg, 
                                            f_add = NPIP.f_add,
                                            constraints = NPIP.constraints)

Found existing BEM file, loading


In [8]:
wec_emp._forces

{'intrinsic_impedance': <function wecopttool.core.force_from_rao_transfer_function.<locals>.force(wec, x_wec, x_opt, waves)>,
 'excitation': <function wecopttool.core.force_from_waves.<locals>.force(wec, x_wec, x_opt, waves)>,
 'Generator': <bound method NonlinearInvertedPendulumPTO.torque_from_PTO of <Pioneer_Inverted_Pendulum.NonlinearInvertedPendulumPTO object at 0x0000019154815250>>,
 'Friction': <bound method NonlinearInvertedPendulumPTO.nonlinear_torque_from_friction of <Pioneer_Inverted_Pendulum.NonlinearInvertedPendulumPTO object at 0x0000019154815250>>,
 'Spring': <bound method NonlinearInvertedPendulumPTO.torque_from_nl_spring of <Pioneer_Inverted_Pendulum.NonlinearInvertedPendulumPTO object at 0x0000019154815250>>,
 'Pendulum': <bound method NonlinearInvertedPendulumPTO.torque_from_pendulum of <Pioneer_Inverted_Pendulum.NonlinearInvertedPendulumPTO object at 0x0000019154815250>>}

In [9]:
results = wec_emp.solve(
    waves_regular,
    obj_fun = NPIP.average_electrical_power,
    nstate_opt = NPIP.nstate_opt,
    optim_options={'maxiter': 500},
    # x_wec_0=x_wec_0, # initialize with result from linear spring case
    # x_opt_0=x_opt_0, # initialize with result from linear spring case
    scale_x_wec=1e1,
    scale_x_opt=np.concatenate((np.array([1e-1])*np.ones(NPIP.nstate_pto), 1e-1 * np.ones(NPIP.nstate_pen))),
    scale_obj=1e-2,
)

C:\Users\dtgaebe\WecOptTool\wecopttool\core.py:756: UserWarning: The `squeeze` kwarg to GroupBy is being removed.Pass .groupby(..., squeeze=False) to disable squeezing, which is the new default, and to silence this warning.
  for realization, wave in waves.groupby('realization'):
INFO:wecopttool.core:Solving pseudo-spectral control problem for realization number 0.
INFO:wecopttool.core:Scaled [max(x_wec), max(x_opt), obj_fun(x)]: [1.20e+00, 1.40e+00, 6.15e+03]
INFO:wecopttool.core:Scaled [max(x_wec), max(x_opt), obj_fun(x)]: [1.17e+00, 1.30e+00, 9.98e+03]
INFO:wecopttool.core:Scaled [max(x_wec), max(x_opt), obj_fun(x)]: [1.26e+00, 1.06e+00, 4.21e+03]
INFO:wecopttool.core:Scaled [max(x_wec), max(x_opt), obj_fun(x)]: [1.19e+00, 5.38e+00, 1.61e+05]
INFO:wecopttool.core:Scaled [max(x_wec), max(x_opt), obj_fun(x)]: [1.04e+00, 4.27e+00, 4.10e+04]
INFO:wecopttool.core:Scaled [max(x_wec), max(x_opt), obj_fun(x)]: [9.82e-01, 1.03e+01, 3.60e+05]
INFO:wecopttool.core:Scaled [max(x_wec), max(x_opt

Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 7850.229322968544
            Iterations: 52
            Function evaluations: 305
            Gradient evaluations: 48


Exception: Positive directional derivative for linesearch    (Exit mode 8)